In [1]:
import numpy as np
import pandas as pd
idx = pd.IndexSlice
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from itertools import product

import sys
sys.path.append('../funcs')
from taus import decorrelation_temporal_model, fit_coh_decay_model

In [129]:
def clean_snotel(df):
    # convert inches to meters
    df['swe'] = df['SWE'] * 0.0254
    df['sd'] = df['SNOWDEPTH'] * 0.0254
    # convert F to C
    df['temp'] = (df['AVG AIR TEMP'] - 32) * 5/9
    df = df.drop(['SWE', 'SWE_units', 'SNOWDEPTH', 'SNOWDEPTH_units', 'AVG AIR TEMP', 'AVG AIR TEMP_units'], axis = 1)
    return df

def MetersToDecimalDegrees(meters, latitude):
    # https://stackoverflow.com/questions/25237356/convert-meters-to-decimal-degrees
    return meters / (111.32 * 1000 * np.cos(latitude * (np.pi / 180)))


In [130]:
dss= {fp.stem.replace('_full', ''): xr.open_dataset(fp) for fp in sorted(list(Path('/Users/rdcrlzh1/Documents/uavsar-coherence/uavsar').glob('*.nc')))}
snotels = {fp.stem: {cf.stem:pd.read_csv(cf, comment='#', index_col=0) for cf in fp.glob('*.csv')} for fp in Path('/Users/rdcrlzh1/Documents/uavsar-coherence/data/snotel/').rglob('*') if fp.stem in dss.keys()}
snotels = {k: {k_s: clean_snotel(v_s) for k_s, v_s in v.items()} for k, v in snotels.items()}
state_abbr = {'Colorado':'CO', 'Idaho': 'ID', 'California':'CA', 'New Mexico': 'NM', 'Utah': 'UT', 'Montana': 'MT'}
state_abbr = {v: k for k, v in state_abbr.items()}
snotel_list = pd.read_csv('/Users/rdcrlzh1/Documents/uavsar-coherence/data/snotel/snotel-list.csv', index_col=['State', 'ID'])
snotel_list.index = snotel_list.index.set_levels(snotel_list.index.levels[1].str.replace('\t', ''), level=1)

# tolerance around each site 100 m in WGS84 at mid latitudes
tol = 0.00090009

In [92]:
def MetersToDecimalDegrees(meters, latitude):
    return meters / (111.32 * 1000 * np.cos(latitude * (np.pi / 180)))


In [112]:
for stem, ds in dss.items():
    diffs = pd.DataFrame(index = pd.MultiIndex(levels=[[],[],[],[]], codes=[[],[],[],[]], names = ['site', 't1', 't2', 'snotel']), columns = ['cor', 'days', 'temp_diff', 'swe_diff', 'sd_diff', 'swe_t1', 'swe_t2', 'sd_t1', 'sd_t2', 'temp_t1','temp_t2'])
    if stem not in snotels.keys(): continue
    loc_snotels = snotels[stem]
    for t1, t2, pol, heading in product(ds.time1.values, ds.time2.values, ds.pol.values, ds.heading.values):
        if (~ds['cor'].sel(time1 = t1, time2 = t2, pol = pol, heading = heading).isnull()).sum() == 0: continue
        cor = ds['cor'].sel(time1 = t1, time2 = t2, pol = pol, heading = heading)
        for snotel_id, snotel_data in loc_snotels.items():
            sid, state, network = snotel_id.split(':')
            snotel_meta = snotel_list.loc[(state_abbr[state], sid)]
            lat, long = snotel_meta['Latitude'], snotel_meta['Longitude']
            # calculate decimal degrees for 50 meters to buffer 100 m box at this latitude
            tol_50m = MetersToDecimalDegrees(50, lat)
            # coherence for 100 meter box centered on snotel
            diffs.loc[(stem, t1, t2, snotel_id), 'cor'] = cor.sel(x = slice(long - tol_50m, long + tol_50m), y = slice(lat + tol_50m, lat - tol_50m)).values.mean()
            diffs.loc[(stem, t1, t2, snotel_id), 'days'] = (t2 - t1).days
            



KeyboardInterrupt: 